- download chrome driver:  https://sites.google.com/chromium.org/driver/

In [1]:
url = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/warminsko--mazurskie/olsztyn/olsztyn/olsztyn'

In [2]:
import os

# prepering directories
path = os.getcwd()
# subfolders
input_dir = os.path.join(path, 'inputs')
output_dir = os.path.join(path, 'outputs')

In [3]:
# selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# BeautifulSoup
from bs4 import BeautifulSoup

import datetime

In [4]:
# chrome driver location
PATH = os.path.join(path, 'inputs','chromedriver.exe' )

In [5]:
# chrom driver configuration
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') # without opening browser

driver = webdriver.Chrome(PATH, options=options)

wait = WebDriverWait(driver, 10)

In [6]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="7aba6b4121e03284369a6c7a618fb4e3")>

In [7]:
# open website using webdriver
driver.get(url)
print(f"Opens website: {url}")

# accepting terms and conditions
# step One - pokaz cele
element = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-pc-btn-handler')))
element.click()
# step two -  confirming preferences and closing the modal window
element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Potwierdzenie moich wyborów')]")))
element.click()

Opens website: https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/warminsko--mazurskie/olsztyn/olsztyn/olsztyn


In [8]:
# step - get number of the paginations
## using SOUP
soup = BeautifulSoup(driver.page_source, 'html.parser')
# save homepage to the file & soup
date = datetime.datetime.now().strftime("%Y%m%d") # current date
with open(os.path.join(path, 'outputs', 'home_page_' + date + '.html'), "w", encoding='utf-8') as file:
    file.write(str(soup))
    
# finds nav with paginations 
# scrapped HTML code can be displayed and pretty printed in Notepad++ using the plugin XML tools
target = soup.find('nav', attrs={'aria-label':'Nawigacja po paginacji'})
date = datetime.datetime.now().strftime("%Y%m%d") # current date
with open(os.path.join(path, 'outputs', 'nav_pagination' + date + '.html'), "w", encoding='utf-8') as file:
    file.write(str(target))
    
# selects all hyperlinks from the nav section
hrefs = target.find_all("a")
# get the hyperlink with the highest number which represents number of pages with adds for the given city
pagination_last = 1
for a in hrefs:
    try:
        num = int(a.get_text())
        pagination_last =  num if pagination_last < num else pagination_last
    except:
        pass
print(f'Number of pages: {pagination_last}')

Number of pages: 21


In [9]:
'''
<div data-cy="search.listing.organic"> container that keeps data with adds - 'Wszystkie ogłoszenia' lets call it all adds 
'''
all_adds_page = soup.find('div', attrs={'data-cy':'search.listing.organic'})
date = datetime.datetime.now().strftime("%Y%m%d") # current date
with open(os.path.join(path, 'outputs', 'all_adds_page_1' + date + '.html'), "w", encoding='utf-8') as file:
    file.write(str(all_adds_page))
    

In [10]:
# OPENS FILE EXPLORER OF THE GIVEN DIRECTORY
import subprocess
subprocess.Popen(f'explorer "{output_dir}"')

In [12]:


def has_article(tag):
    return tag.name == 'li' and tag.find('article') is not None

# Use find_all with the custom function
# this can be wrote with lambda
lis = all_adds_page.find_all(has_article)

len(lis)

36

In [ ]:
with open(os.path.join(path, 'outputs', 'add_first' + date + '.html'), "w", encoding='utf-8') as file:
    file.write(str(lis[0]))

In [91]:
# DATA STRUCTURE
adds = []
for i, add in enumerate(lis):
    artcile = add.find('article')
    # gets any tags with the attribute 'title'
    titles = artcile.find_all(lambda tag: tag is not None and tag.has_attr("title"))
    price = price_meter = rooms = area = more = None
    
    info = add.find_all('span')
    for e in info:
        e = e.text
        if 'zł' in e and 'zł/m²' not in e:
            price = e
        if 'zł/m²' in e:
            price_meter = e
        if [element for element in ['pokoje','pokoi','pokój'] if(element in e)]:
            rooms = e
        if 'm²' in e and 'zł/m²' not in e:
            area = e
            
    more = info[-1].text
            
    adds.append({
        'free_text': titles[0]['title'],
        'address': titles[1]['title'],
        'price': price,
        'price_meter': price_meter,
        'rooms': rooms,
        'area': area,
        'more':more,
        'add_link': add.select_one('a')['href']
    })

In [92]:
import pandas as pd
 
# Initialize data to lists.
data = [{'a': 1, 'b': 2, 'c': 3},
        {'a': 10, 'b': 20, 'c': 30}]
 
# Creates DataFrame.
df = pd.DataFrame(adds)
df.head(2)

,free_text,address,price,price_meter,rooms,area,more,add_link
0,Przestronne 3pokojowe mieszkanie w Centrum Jarot!,Mieszkanie na sprzedaż: ul. Floriana Piotrowsk...,630 000 zł,9251 zł/m²,3 pokoje,"68,1 m²",Biuro nieruchomości,/pl/oferta/przestronne-3pokojowe-mieszkanie-w-...
1,Mieszkanie 3 pokoje Pieczewo Olsztyn,Mieszkanie na sprzedaż: ul. Władysława Gębika ...,549 000 zł,8318 zł/m²,3 pokoje,66 m²,Biuro nieruchomości,/pl/oferta/mieszkanie-3-pokoje-pieczewo-olszty...


In [90]:
df['more'].unique()

array(['DOM NIERUCHOMOŚCI', 'PROFiN Finanse i Nieruchomości', '3 pokoje',
       'ARBET Investment Group sp. z o.o.', 'Benewent Sp. z o.o.',
       'WISDOM Nieruchomości', 'Dunajscy Nieruchomości',
       'Galeria Nieruchomości Hanna Tetmer-Rzepecka',
       'Valor Nieruchomości i Kredyty Hipoteczne', 'NewHouse',
       'Olsztyńskie Centrum Nieruchomości s.c.',
       'Limeno - Salon Rynku Pierwotnego',
       'PERFEKT Biuro Nieruchomości S.C. Ewa Andrzejczuk,Elżbieta Rink',
       '2 pokoje', 'Chata w Kwadracie', 'HONESTA',
       'Warmia Brokers Lidia Barasińska', 'TreeHouse Nieruchomości',
       'Freedom Nieruchomości | Biura'], dtype=object)